## LVXNN

In [1]:
import time
import numpy as np
import pandas as pd 
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics.pairwise import cosine_similarity
from gaminet.utils import local_visualize
from gaminet.utils import global_visualize_density
from gaminet.utils import feature_importance_visualize
from gaminet.utils import plot_trajectory
from gaminet.utils import plot_regularization
from sklearn.linear_model import LinearRegression
from collections import OrderedDict
import sys
sys.path.append('../../')
from lvxnn.LVXNN import LV_XNN
from lvxnn.DataReader import data_initialize

data= pd.read_csv('data/train.csv')
train , test = train_test_split(data,test_size=0.2 ,random_state=2)

list1 = data.columns
meta_info = OrderedDict()

for i in list1:
    meta_info[i]={'type': 'categorical','source':'user'}
meta_info['income']={"type":"continues",'source':'user'}
meta_info['cust_seniority']={"type":"continues",'source':'user'}
meta_info['age'] = {"type":"continues",'source':'user'}
meta_info['item'] = {'type': 'categorical','source':'item'}
meta_info['user_id']={"type":"id",'source':'user'}
meta_info['item_id']={"type":"id",'source':'item'}
meta_info['target']={"type":"target",'source':''}


tr_x, tr_Xi, tr_y , te_x , te_Xi, te_y, meta_info, model_info = data_initialize(train,test,meta_info,"Classification")

def auto_test():
    cold_auc = []
    cold_logloss = []
    warm_auc = []
    warm_logloss = []
    gami_auc = []
    gami_logloss = []

    for times in range(10):
        
        print(times)


        model = LV_XNN(model_info=model_info, meta_info=meta_info, subnet_arch=[8, 16],interact_arch=[20, 10],activation_func=tf.tanh, batch_size=1000, lr_bp=0.01, auto_tune=False,
               interaction_epochs=20,main_effect_epochs=20,tuning_epochs=10,loss_threshold_main=0.01,loss_threshold_inter=0.01,alpha=0,
              verbose=True,val_ratio=0.125, early_stop_thres=100,interact_num=10,u_group_num=10,i_group_num=20,scale_ratio=0.8,n_power_iterations=5,n_oversamples=0,
              mf_training_iters=1,mf_tuning_iters=100,change_mode=True,convergence_threshold=0.001,max_rank=3,shrinkage_value=20, epsilon=0,random_state=times,combine_range=0.95)
    
        st_time = time.time()
        model.fit(tr_x,tr_Xi, tr_y)
        ed_time = time.time()
        
        pred = model.predict(te_x, te_Xi)
        
        cold_y = te_y[(te_Xi[:,1] == 'cold') | (te_Xi[:,0] == 'cold')]
        cold_pred = pred[(te_Xi[:,1] == 'cold') | (te_Xi[:,0] == 'cold')]
        warm_y = te_y[(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')]
        warm_pred = pred[(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')]
    
        cold_auc.append(roc_auc_score(cold_y,cold_pred))
        cold_logloss.append(log_loss(cold_y,cold_pred))
        warm_auc.append(roc_auc_score(warm_y,warm_pred))
        warm_logloss.append(log_loss(warm_y,warm_pred))
        gami_auc.append(roc_auc_score(te_y,model.final_gam_model.predict(te_x)))
        gami_logloss.append(log_loss(te_y,model.final_gam_model.predict(te_x)))
        
    i_result = np.array([np.mean(cold_auc),np.mean(cold_logloss),np.mean(warm_auc),np.mean(warm_logloss)]).reshape(1,-1)
    result = pd.DataFrame(i_result,columns=['cold_auc','cold_logloss','warm_auc','warm_logloss'])
    
    g_result = np.array([np.mean(gami_auc),np.mean(gami_logloss)]).reshape(1,-1)
    g_result = pd.DataFrame(g_result,columns=['auc','logloss'])
    
    return result, g_result

results, g_result = (auto_test())
results.to_csv('result/LVXNN_result.csv',index=None)
g_result.to_csv('result/gami_result.csv',index=None)

Memory usage of dataframe is 108.45 MB


../..\lvxnn\DataReader.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.int8)
../..\lvxnn\DataReader.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.int16)
../..\lvxnn\DataReader.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].asty

Memory usage after optimization is: 22.80 MB
Decreased by 79.0%
Memory usage of dataframe is 27.11 MB
Memory usage after optimization is: 5.70 MB
Decreased by 79.0%
cold start user: 13104
cold start item: 9
0
ListWrapper(['ind_empleado', 'sex', 'ind_nuevo', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'ind_actividad_cliente', 'segmento', 'pais_residencia', 'canal_entrada', 'age', 'cust_seniority', 'income', 'marriage', 'item'])
####################GAMI-Net training start.####################
##########Stage 1: main effect training start.##########
Main effects training epoch: 1, train loss: 0.16054, val loss: 0.16017
Main effects training epoch: 2, train loss: 0.16036, val loss: 0.16004
Main effects training epoch: 3, train loss: 0.16008, val loss: 0.15986
Main effects training epoch: 4, train loss: 0.15988, val loss: 0.15960
Main effects training epoch: 5, train loss: 0.15967, val loss: 0.15946
Main effects training epoch: 6, train loss: 0.15980, val loss: 0

C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


[SoftImpute] Iter 1: observed BCE=0.059762 validation BCE=0.128003,rank=3
[SoftImpute] Iter 2: observed BCE=0.058335 validation BCE=0.129750,rank=3
[SoftImpute] Iter 3: observed BCE=0.057240 validation BCE=0.132436,rank=3
[SoftImpute] Iter 4: observed BCE=0.056207 validation BCE=0.134527,rank=3
[SoftImpute] Stopped after iteration 4 for lambda=1.589555
final num of user group: 5
final num of item group: 6
change mode state : True
time cost: 5.309386968612671
After the matrix factor stage, training error is 0.05621, validation error is 0.13453
1
ListWrapper(['ind_empleado', 'sex', 'ind_nuevo', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'ind_actividad_cliente', 'segmento', 'pais_residencia', 'canal_entrada', 'age', 'cust_seniority', 'income', 'marriage', 'item'])
####################GAMI-Net training start.####################
##########Stage 1: main effect training start.##########
Main effects training epoch: 1, train loss: 0.16051, val loss: 0.16018
Main e

...

[SoftImpute] Iter 1: observed BCE=0.060703 validation BCE=0.130465,rank=3


C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


[SoftImpute] Iter 2: observed BCE=0.058366 validation BCE=0.132802,rank=3
[SoftImpute] Iter 3: observed BCE=0.060191 validation BCE=0.136552,rank=3
[SoftImpute] Iter 4: observed BCE=0.059967 validation BCE=0.137879,rank=3
[SoftImpute] Stopped after iteration 4 for lambda=1.590409
final num of user group: 7
final num of item group: 6
change mode state : True
time cost: 5.529951810836792
After the matrix factor stage, training error is 0.05997, validation error is 0.13788
9
ListWrapper(['ind_empleado', 'sex', 'ind_nuevo', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'ind_actividad_cliente', 'segmento', 'pais_residencia', 'canal_entrada', 'age', 'cust_seniority', 'income', 'marriage', 'item'])
####################GAMI-Net training start.####################
##########Stage 1: main effect training start.##########
Main effects training epoch: 1, train loss: 0.16064, val loss: 0.15932
Main effects training epoch: 2, train loss: 0.16035, val loss: 0.15901
Main effe

C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\64161\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


[SoftImpute] Iter 1: observed BCE=0.059794 validation BCE=0.128447,rank=3
[SoftImpute] Iter 2: observed BCE=0.061279 validation BCE=0.129281,rank=3
[SoftImpute] Iter 3: observed BCE=0.060511 validation BCE=0.131100,rank=3
[SoftImpute] Iter 4: observed BCE=0.062672 validation BCE=0.132611,rank=3
[SoftImpute] Stopped after iteration 4 for lambda=1.585256
final num of user group: 6
final num of item group: 7
change mode state : True
time cost: 6.35776424407959
After the matrix factor stage, training error is 0.06267, validation error is 0.13261


## xgboost

In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss

data= pd.read_csv('data/train.csv')
data = data.drop(['item'],1)
train , test = train_test_split(data,test_size=0.2, random_state=2)
data,val = train_test_split(train,test_size=0.125)

x=data.iloc[:,:-1].values
y=data.iloc[:,-1].values
x_t = test.iloc[:,:-1].values
y_t = test.iloc[:,-1].values

enc = MinMaxScaler()
x = enc.fit_transform(x)
x_t = enc.fit_transform(x_t)

def auto_test():
    auc = []
    logloss = []
    for times in range(10):
        xgb = XGBClassifier(n_jobs=-1)
        xgb.fit(x,y)
        pred = xgb.predict_proba(x_t)
        
        auc.append(roc_auc_score(y_t,pred[:,1]))
        logloss.append(log_loss(y_t,pred[:,1]))

    i_result = np.array([np.mean(auc),np.mean(logloss)]).reshape(1,-1)
    result = pd.DataFrame(i_result,columns=['auc','logloss'])
    
    return result

results = (auto_test())
results.to_csv('result/xgboost_result.csv',index=None)

## SVD

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss

data= pd.read_csv('data/train.csv')
data = data.drop(['item'],1)
train , test = train_test_split(data,test_size=0.2,random_state=2)
data,val = train_test_split(train,test_size=0.125)

Xi = data.iloc[:,-3:-1].values
y = data.iloc[:,-1].values
Xi_t = test.iloc[:,-3:-1].values
y_t = test.iloc[:,-1].values

tr_ratings_dict = {'itemID': Xi[:,1].tolist(),
                'userID': Xi[:,0].tolist(),
                'rating': y.tolist()}

tr_df = pd.DataFrame(tr_ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(y.min(), y.max()))

# The columns must correspond to user id, item id and ratings (in that order).
tr_data = Dataset.load_from_df(tr_df[['userID', 'itemID', 'rating']], reader)

tr_data = tr_data.build_full_trainset()

def auto_test():
    auc = []
    logloss = []
    for j in range(10):
        model = SVD(n_factors=3)

        model.fit(tr_data)

        pred = []
        
        for i in range(Xi_t.shape[0]):
            pred.append(model.predict(Xi_t[i,0],Xi_t[i,1],Xi_t[i,0]).est)
    
        pred2 = np.array(pred).ravel()

        auc.append(roc_auc_score(y_t,pred2))
        logloss.append(log_loss(y_t,pred2))
    
    i_result = np.array([np.mean(auc),np.mean(logloss)]).reshape(1,-1)
    result = pd.DataFrame(i_result,columns=['auc','logloss'])
    
    return result

results = (auto_test())
results.to_csv('result/svd_result.csv',index=None)

## deepfm

In [2]:
import pandas as pd
data = pd.read_csv('data/train.csv')

In [14]:
class config():
# set the path-to-files
    
    TRAIN_FILE = "./data/train.csv"
    SUB_DIR = "./result"
    NUM_SPLITS = 3
    RANDOM_SEED = 2017

# types of columns of the dataset dataframe
    CATEGORICAL_COLS = ["item","ind_empleado","sex","indrel_1mes",'tiprel_1mes','indresi','indext','indfall','ind_actividad_cliente','segmento','pais_residencia','canal_entrada']
    NUMERIC_COLS = ["ind_nuevo", "indrel", "age", "cust_seniority"]
    IGNORE_COLS = ["target"]
    
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
sys.path.append('../benchmark/deepfm/')
from DataReader import FeatureDictionary, DataParser
from DeepFM import DeepFM


def _load_data():

    dfTrain = pd.read_csv(config.TRAIN_FILE)
    dfTrain , dfTest = train_test_split(dfTrain,test_size=0.2,random_state=2)

    def preprocess(df):        
        cols = [c for c in df.columns if c not in ["target"]]
        return df

    dfTrain = preprocess(dfTrain)
    dfTest = preprocess(dfTest)
    cols = [c for c in dfTrain.columns if c not in ["target"]]


    X_train = dfTrain[cols].values
    y_train = dfTrain["target"].values
    X_test = dfTest[cols].values
    
    ids_test = dfTest["user_id"].values
    idv_test = dfTest["item_id"].values
    y_test = dfTest['target'].values
        
    return dfTrain, dfTest, X_train, y_train, X_test, ids_test,idv_test, y_test


def _run_base_model_dfm(dfTrain, dfTest, folds, dfm_params):
    fd = FeatureDictionary(dfTrain=dfTrain, dfTest=dfTest,
                           numeric_cols=config.NUMERIC_COLS,
                           ignore_cols=config.IGNORE_COLS)
    data_parser = DataParser(feat_dict=fd)
    Xi_train, Xv_train, y_train = data_parser.parse(df=dfTrain, has_label=True)
    Xi_test, Xv_test, ids_test,idv_test = data_parser.parse(df=dfTest)
    
    dfm_params["feature_size"] = fd.feat_dim
    dfm_params["field_size"] = len(Xi_train[0])
    print(dfm_params)

    y_train_meta = np.zeros((dfTrain.shape[0], 1), dtype=float)
    y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)
    _get = lambda x, l: [x[i] for i in l]
    gini_results_cv = np.zeros(len(folds), dtype=float)
    gini_results_epoch_train = np.zeros((len(folds), dfm_params["epoch"]), dtype=float)
    gini_results_epoch_valid = np.zeros((len(folds), dfm_params["epoch"]), dtype=float)
    for i, (train_idx, valid_idx) in enumerate(folds):
        Xi_train_, Xv_train_, y_train_ = _get(Xi_train, train_idx), _get(Xv_train, train_idx), _get(y_train, train_idx)
        Xi_valid_, Xv_valid_, y_valid_ = _get(Xi_train, valid_idx), _get(Xv_train, valid_idx), _get(y_train, valid_idx)

        dfm = DeepFM(**dfm_params)
        dfm.fit(Xi_train_, Xv_train_, y_train_, Xi_valid_, Xv_valid_, y_valid_)

        y_train_meta[valid_idx,0] = dfm.predict(Xi_valid_, Xv_valid_)
        y_test_meta[:,0] += dfm.predict(Xi_test, Xv_test)
        
        gini_results_cv[i] = mean_absolute_error(y_valid_, y_train_meta[valid_idx])
        gini_results_epoch_train[i] = dfm.train_result
        gini_results_epoch_valid[i] = dfm.valid_result

    y_test_meta /= float(len(folds))

    # save result
    return y_train_meta, y_test_meta

# load data
dfTrain, dfTest, X_train, y_train, X_test, ids_test ,idv_test, y_test= _load_data()

# folds
folds = list(KFold(n_splits=config.NUM_SPLITS, shuffle=True,
                             random_state=config.RANDOM_SEED).split(X_train, y_train))



# ------------------ DeepFM Model ------------------
# params
dfm_params = {
    "embedding_size": 3,
    "deep_layers": [32, 32],
    "use_deep" : True ,
    "use_fm" : True , 
    "deep_layers_activation": tf.nn.relu,
    "loss_type" : "logloss",
    "epoch": 10 ,
    "batch_size": 1024,
    "learning_rate": 0.01,
    "optimizer_type": "adam",
    "batch_norm": 0,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.1,
    "verbose": True,
    "eval_metric": log_loss,
    "random_seed": config.RANDOM_SEED
}

def auto_test(deep):
    auc = []
    logloss = []
    dfm_params['use_deep']=deep
    
    for i in range(10):
        y_train_dfm, y_test_dfm = _run_base_model_dfm(dfTrain, dfTest, folds, dfm_params)
        auc.append(roc_auc_score(y_test,y_test_dfm))
        logloss.append(log_loss(y_test,y_test_dfm))
    
    i_result = np.array([np.mean(auc),np.mean(logloss)]).reshape(1,-1)
    results = pd.DataFrame(i_result,columns=['auc','logloss'])
    
    return results
result_1 = (auto_test(True))
result_2 = (auto_test(False))
result_1.to_csv('result/deepfm_result.csv',index=None)
result_2.to_csv('result/fm_result.csv',index=None)

{'embedding_size': 3, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000014B8672B598>, 'loss_type': 'logloss', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.1, 'verbose': True, 'eval_metric': <function log_loss at 0x0000014B9A90E1E0>, 'random_seed': 2017, 'feature_size': 64139, 'field_size': 20}
#params: 259620
[1] train-result=0.1544, valid-result=0.1696 [10.5 s]
[2] train-result=0.1256, valid-result=0.1763 [9.6 s]
[3] train-result=0.1079, valid-result=0.1849 [9.2 s]
[4] train-result=0.0819, valid-result=0.1988 [9.4 s]
[5] train-result=0.0567, valid-result=0.2208 [9.6 s]
[6] train-result=0.0422, valid-result=0.2413 [11.1 s]
[7] train-result=0.0347, valid-result=0.2520 [11.0 s]


C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[8] train-result=0.0294, valid-result=inf [10.0 s]


C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[9] train-result=0.0252, valid-result=nan [10.3 s]


C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Users\64161\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[10] train-result=nan, valid-result=nan [10.0 s]
#params: 259620


KeyboardInterrupt: 